# Evn

In [ ]:
# imports
import argparse
import os
import random
import shutil
import json
import zipfile
import math
import copy
import collections
import re

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sentencepiece as spm
import tensorflow as tf
import tensorflow.keras.backend as K

from tqdm.notebook import tqdm, trange

In [ ]:
# 환경 설정
args = {
    # random seed value
    "seed": 1234
}
args = argparse.Namespace(**args)

print(args)

In [ ]:
# random seed 설정
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

In [ ]:
# gpu 사용량 확인
!nvidia-smi

In [ ]:
# data dir (환경에 맞게 수정하세요.)
data_dir = '/content/drive/MyDrive/문서/강의계획서/삼성전기/삼성전기.20220228/data'
os.listdir(data_dir)

# Vocabulary*

In [ ]:
# vocab loading
vocab = spm.SentencePieceProcessor()
vocab.load(os.path.join(data_dir, 'kowiki', 'kowiki_32000.model'))

# Padding

In [ ]:
# 말뭉치
corpus = """나는 학생 입니다
당신은 수학 선생님 입니다"""

In [ ]:
# 줄 바꿈 단위로 입력 문장 tokenize
inputs = []
for line in corpus.split("\n"):
    inputs.append(vocab.encode_as_pieces(line))
inputs

In [ ]:
# token을 일련번호로
input_ids = []
for line in inputs:
    input_ids.append(vocab.piece_to_id(line))
input_ids

In [ ]:
# 일련번호를 한줄씩 numpy array로
for line in input_ids:
    display(np.array(line))

In [ ]:
# 일련번호 전체를 numpy array로
np.array(input_ids)

In [ ]:
# 최대 길이
max_len = 0
for line in input_ids:
    max_len = max(max_len, len(line))
max_len

In [ ]:
# 최대 길이로 맞게 뒤에 [PAD] 추가후 전체를 numpy array로
pad_input_ids = []
for line in input_ids:
    line = line[:max_len]  # 긴 입력은 자름
    line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
    pad_input_ids.append(line)
np.array(pad_input_ids)

In [ ]:
# 최대 길이로 맞게 뒤에 [PAD] 추가후 전체를 numpy array로
max_len = 3
pad_input_ids = []
for line in input_ids:
    line = line[:max_len]  # 긴 입력은 자름
    line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
    pad_input_ids.append(line)
np.array(pad_input_ids)

In [ ]:
# 최대 길이로 맞게 뒤에 [PAD] 추가후 전체를 numpy array로
max_len = 10
pad_input_ids = []
for line in input_ids:
    line = line[:max_len]  # 긴 입력은 자름
    line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
    pad_input_ids.append(line)
np.array(pad_input_ids)

# Padding Strategy

In [ ]:
# 말뭉치
corpus = """수학은 수식이 복잡해서 어렵다
수학은 공식이 어렵다
수학은 수식이 이해되면 쉽다
숙학은 공식이 능통하면 쉽다 영어는 단어가 어렵다
영어는 듣기가 복잡해서 어렵다 영어는 단어가 이해되면 쉽다
영어는 듣기가 능통하면 쉽다 국어는 지문이 어렵다
국어는 한문이 많아서 어렵다 국어는 한문이 능통하면 쉽다
국어는 지문이 이해되면 쉽다 수학은 공식이 쉽다
수학은 수식이 복잡해서 어렵다 영어는 듣기가 능통하면 쉽다 국어는 지문이 이해되면 쉽다 한문이 어렵다"""

In [ ]:
# 줄 바꿈 단위로 입력 문장 tokenize
inputs = []
for line in corpus.split("\n"):
    inputs.append(vocab.encode_as_pieces(line))
inputs

In [ ]:
# token을 일련번호로
input_ids = []
for line in inputs:
    input_ids.append(vocab.piece_to_id(line))
input_ids

In [ ]:
# 최대 길이
max_len = 0
for line in input_ids:
    max_len = max(max_len, len(line))
max_len

In [ ]:
# 최대 길이로 맞게 뒤에 [PAD] 추가후 전체를 numpy array로
pad_input_ids = []
for line in input_ids:
    line = line[:max_len]  # 긴 입력은 자름
    line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
    pad_input_ids.append(line)
np_input_ids = np.array(pad_input_ids)
np_input_ids.shape, np_input_ids

In [ ]:
# 전체 token 수
total = np_input_ids.shape[0] * np_input_ids.shape[1]
total

In [ ]:
# pad 비율
n_pad = np.sum(np_input_ids == 0)
n_pad, n_pad / total

In [ ]:
# 최대 길이로 맞게 뒤에 [PAD] 추가후 전체를 numpy array로
max_len = 15  # 길이가 긴 문장을 버림
pad_input_ids = []
for line in input_ids:
    line = line[:max_len]  # 긴 입력은 자름
    line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
    pad_input_ids.append(line)
np_input_ids = np.array(pad_input_ids)
np_input_ids.shape, np_input_ids

In [ ]:
# 전체 token 수
total = np_input_ids.shape[0] * np_input_ids.shape[1]
# pad 비율
n_pad = np.sum(np_input_ids == 0)
total, n_pad, n_pad / total

In [ ]:
# batch 단위로 처리
n_batch = 3
for i in range(0, len(input_ids), n_batch):
    batch_input = input_ids[i: i + n_batch]
    max_len = max([len(line) for line in batch_input])
    pad_input_ids = []
    for line in batch_input:
        line = line[:max_len]  # 긴 입력은 자름
        line += [0] * (max_len - len(line))  # 짧은 입력은 [PAD] 추가
        pad_input_ids.append(line)
    display(np.array(pad_input_ids))
    print()

# 실습
- data_dir / kowiki / kowiki.txt.zip의 '지미카터' 문서를 encode_as_ids를 이용해 줄단위로 tokenize 해서 배열에 저장해 보세요.
- 각 줄의 토큰 개수를 출력해 보세요. 최대, 최소, 평균을 구하세요.
- 가장 긴 token에 맞게 pad(0)을 추가해서 numpy array로 변경해 보세요.
- 평균에 맞게 token의 길이를 자르거나 pad(0)을 추가해서 numpy array로 변경해 보세요.